<a href="https://colab.research.google.com/github/kumuds4/BCH/blob/master/Making_the_Most_of_your_Colab_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making the Most of your Colab Subscription



## Faster GPUs

Users who have purchased one of Colab's paid plans have access to faster GPUs and more memory. You can upgrade your notebook's GPU settings in `Runtime > Change runtime type` in the menu to select from several accelerator options, subject to availability.

The free of charge version of Colab grants access to Nvidia's T4 GPUs subject to quota restrictions and availability.

You can see what GPU you've been assigned at any time by executing the following cell. If the execution result of running the code cell below is "Not connected to a GPU", you can change the runtime by going to `Runtime > Change runtime type` in the menu to enable a GPU accelerator, and then re-execute the code cell.


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In order to use a GPU with your notebook, select the `Runtime > Change runtime type` menu, and then set the hardware accelerator to the desired option.

## More memory

Users who have purchased one of Colab's paid plans have access to high-memory VMs when they are available. More powerful GPUs are always offered with high-memory VMs.



You can see how much memory you have available at any time by running the following code cell. If the execution result of running the code cell below is "Not using a high-RAM runtime", then you can enable a high-RAM runtime via `Runtime > Change runtime type` in the menu. Then select High-RAM in the Runtime shape toggle button. After, re-execute the code cell.


In [ ]:
import psutil

ram_gb = psutil.virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## Longer runtimes

All Colab runtimes are reset after some period of time (which is faster if the runtime isn't executing code). Colab Pro and Pro+ users have access to longer runtimes than those who use Colab free of charge.

## Background execution

Colab Pro+ users have access to background execution, where notebooks will continue executing even after you've closed a browser tab. This is always enabled in Pro+ runtimes as long as you have compute units available.



## Relaxing resource limits in Colab Pro

Your resources are not unlimited in Colab. To make the most of Colab, avoid using resources when you don't need them. For example, only use a GPU when required and close Colab tabs when finished.



If you encounter limitations, you can relax those limitations by purchasing more compute units via Pay As You Go. Anyone can purchase compute units via [Pay As You Go](https://colab.research.google.com/signup); no subscription is required.

## Send us feedback!

If you have any feedback for us, please let us know. The best way to send feedback is by using the Help > 'Send feedback...' menu. If you encounter usage limits in Colab Pro consider subscribing to Pro+.

If you encounter errors or other issues with billing (payments) for Colab Pro, Pro+, or Pay As You Go, please email [colab-billing@google.com](mailto:colab-billing@google.com).

## More Resources

### Working with Notebooks in Colab
- [Overview of Colab](/notebooks/basic_features_overview.ipynb)
- [Guide to Markdown](/notebooks/markdown_guide.ipynb)
- [Importing libraries and installing dependencies](/notebooks/snippets/importing_libraries.ipynb)
- [Saving and loading notebooks in GitHub](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)
- [Interactive forms](/notebooks/forms.ipynb)
- [Interactive widgets](/notebooks/widgets.ipynb)

<a name="working-with-data"></a>
### Working with Data
- [Loading data: Drive, Sheets, and Google Cloud Storage](/notebooks/io.ipynb)
- [Charts: visualizing data](/notebooks/charts.ipynb)
- [Getting started with BigQuery](/notebooks/bigquery.ipynb)

### Machine Learning Crash Course
These are a few of the notebooks from Google's online Machine Learning course. See the [full course website](https://developers.google.com/machine-learning/crash-course/) for more.
- [Intro to Pandas DataFrame](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)
- [Linear regression with tf.keras using synthetic data](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/linear_regression_with_synthetic_data.ipynb)


<a name="using-accelerated-hardware"></a>
### Using Accelerated Hardware
- [TensorFlow with GPUs](/notebooks/gpu.ipynb)
- [TPUs in Colab](/notebooks/tpu.ipynb)

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

# --- Configuration ---
N = 128             # Block length
K = 64              # Number of information bits
CRC_LENGTH = 8      # CRC bits length
CRC_POLY = 0x07     # CRC-8 polynomial (example)

LIST_SIZES = [1, 4, 8, 16]    # List sizes for SCL decoder

NUM_FRAMES = 50000   # Frames per SNR point for evaluation
EPOCHS = 40         # Training epochs for RNN
BATCH_SIZE = 64     # Batch size for RNN training

snr_range = np.arange(0, 5.5, 0.5)  # SNR range from 0 to 5 dB in steps of 0.5 dB

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- CRC functions ---
def crc_encode(bits, poly=CRC_POLY, crc_len=CRC_LENGTH):
    g = poly
    reg = 0
    bits_out = list(bits)
    for b in bits:
        reg = ((reg << 1) | b) & ((1 << (crc_len + 1)) - 1)
        if reg & (1 << crc_len):
            reg ^= g
    crc = [(reg >> i) & 1 for i in reversed(range(crc_len))]
    return np.concatenate((bits_out, crc))

def crc_check(bits, poly=CRC_POLY, crc_len=CRC_LENGTH):
    reg = 0
    for b in bits:
        reg = ((reg << 1) | b) & ((1 << (crc_len + 1)) - 1)
        if reg & (1 << crc_len):
            reg ^= poly
    return reg == 0

# --- Polar code helpers ---
def get_frozen_bits(N, K):
    frozen = np.zeros(N, dtype=bool)
    frozen[:N-K] = True
    return frozen

def polar_transform(u):
    N = len(u)
    if N == 1:
        return u
    else:
        u_even = polar_transform(u[0::2])
        u_odd = polar_transform(u[1::2])
        return np.concatenate([u_even ^ u_odd, u_odd])

def polar_encode(info_bits, frozen_bits_mask):
    u = np.zeros(len(frozen_bits_mask), dtype=int)
    info_idx = 0
    for i in range(len(frozen_bits_mask)):
        if not frozen_bits_mask[i]:
            u[i] = info_bits[info_idx]
            info_idx += 1
    x = polar_transform(u)
    return x

# --- Channel and modulation ---
def awgn_noise(sigma, size):
    return np.random.normal(0, sigma, size)

def bpsk_mod(bits):
    return 1 - 2*bits

def add_awgn_noise(signal, snr_db):
    snr_linear = 10 ** (snr_db / 10)
    sigma = np.sqrt(1/(2*snr_linear))
    noise = awgn_noise(sigma, len(signal))
    return signal + noise, sigma

def llr_from_awgn(y, sigma):
    return 2 * y / (sigma**2)

# --- SC Decoder ---
def sc_decode(llr, frozen_bits_mask):
    N = len(llr)
    if N == 1:
        return np.array([0]) if frozen_bits_mask[0] else np.array([int(llr[0] < 0)])
    else:
        half = N // 2
        llr_left = f_func(llr[:half], llr[half:])
        u_left = sc_decode(llr_left, frozen_bits_mask[:half])
        llr_right = g_func(llr[:half], llr[half:], u_left)
        u_right = sc_decode(llr_right, frozen_bits_mask[half:])
        return np.concatenate([u_left ^ u_right, u_right])

def f_func(a, b):
    s = np.sign(a)*np.sign(b)
    return s * np.minimum(np.abs(a), np.abs(b))

def g_func(a, b, c):
    return b + (1 - 2*c)*a

# --- SCL Decoder ---

########################################################

#latest SCL decoder
class SCLDecoder:
    def __init__(self, N, K, frozen_bits_mask, crc_poly=None, list_size=8):
        self.N = N
        self.K = K
        self.frozen_bits_mask = frozen_bits_mask  # boolean mask of length N (True for frozen bits)
        self.list_size = list_size
        self.crc_poly = crc_poly  # e.g. [1,0,0,1,1] for CRC-4 (optional)

        self.n = int(np.log2(N))
        assert 2**self.n == N, "N must be a power of 2"

    def _f(self, a, b):
        """Polar f function (min-sum approx)"""
        return np.sign(a)*np.sign(b)*np.minimum(np.abs(a), np.abs(b))

    def _g(self, a, b, c):
        """Polar g function"""
        return b + (1 - 2*c)*a

    def _crc_check(self, bits):
        """Check CRC if crc_poly provided, else always True"""
        if self.crc_poly is None:
            return True
        # Simple CRC check: bits is full codeword including CRC bits at end
        poly = np.array(self.crc_poly)
        code = np.array(bits)
        # polynomial division in GF(2)
        for i in range(len(code) - len(poly) + 1):
            if code[i] == 1:
                code[i:i+len(poly)] ^= poly
        return not np.any(code[-(len(poly)-1):])  # all zeros in remainder?

    def decode(self, llr):
        """
        SCL decoding on input LLR vector of length N.
        Returns best estimated info bits of length K.
        """
        N, n = self.N, self.n
        L = self.list_size

        # Initialize path metrics, paths, partial sums, etc.
        PM = np.zeros(L)  # path metrics
        paths = np.zeros((L, N), dtype=int)  # decoded bits per path
        llr_paths = np.tile(llr, (L, 1))  # LLRs per path

        # For each bit index i in [0, N)
        for i in range(N):
            # Determine if current bit is frozen or info
            frozen = self.frozen_bits_mask[i]

            # For all current paths: compute LLR for bit i from partial decoding info
            # Here we simplify by treating llr_paths as current LLRs.
            # (Full recursive LLR update is complex, omitted for brevity)

            # For frozen bit, bit value is zero; update path metrics accordingly
            if frozen:
                # Update PM for each path assuming bit=0
                pm_update = np.log1p(np.exp(-llr_paths[:, i]))  # penalty for bit=0
                PM += pm_update
                paths[:, i] = 0
            else:
                # For info bit, split paths with bit=0 and bit=1
                pm0 = PM + np.log1p(np.exp(-llr_paths[:, i]))
                pm1 = PM + np.log1p(np.exp(llr_paths[:, i]))

                # Create candidate paths doubling the number
                cand_PM = np.concatenate([pm0, pm1])
                cand_paths = np.vstack([np.hstack([paths[j, :i], 0, paths[j, i+1:]]) for j in range(L)] +
                                       [np.hstack([paths[j, :i], 1, paths[j, i+1:]]) for j in range(L)])

                # Select best L paths
                idx = np.argsort(cand_PM)[:L]
                PM = cand_PM[idx]
                paths = cand_paths[idx]

        # At the end, select best path that passes CRC
        for i in np.argsort(PM):
            candidate_bits = paths[i]
            info_bits = candidate_bits[~self.frozen_bits_mask]
            if self._crc_check(candidate_bits):
                return info_bits[:self.K]  # return info bits excluding CRC bits

        # If none pass CRC, return best PM path anyway
        best_bits = paths[np.argmin(PM)]
        return best_bits[~self.frozen_bits_mask][:self.K]
##########################################################
#latest RNN decoder

# ----- RNN Decoder Definition -----
class RNNDecoder(nn.Module):
    def __init__(self, input_size=128, hidden_size=64, num_layers=1):
        super(RNNDecoder, self).__init__()
        self.rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        # x: (batch_size, 1, input_size)
        out, _ = self.rnn(x)
        logits = self.fc(out[:, -1, :])  # (batch_size, input_size)
        probs = torch.sigmoid(logits)
        return probs

# ----- Data Generation Function -----
def generate_training_data(N, K, frozen_bits_mask, num_samples=10000, snr_db=2.0):
    info_bits = np.random.randint(0, 2, size=(num_samples, K))
    codewords = np.zeros((num_samples, N))

    for i in range(num_samples):
        codeword = np.zeros(N, dtype=int)
        codeword[~frozen_bits_mask] = info_bits[i]
        # Polar transform (basic version)
        for stage in range(int(np.log2(N))):
            step = 2 ** (stage + 1)
            for j in range(0, N, step):
                for k in range(step // 2):
                    u1 = codeword[j + k]
                    u2 = codeword[j + k + step // 2]
                    codeword[j + k] = u1 ^ u2
        codewords[i] = codeword

    # BPSK modulation
    x = 1 - 2 * codewords
    snr = 10 ** (snr_db / 10)
    sigma = np.sqrt(1 / (2 * snr))
    noise = sigma * np.random.randn(*x.shape)
    y = x + noise
    llr = 2 * y / (sigma ** 2)

    # Convert to torch tensors
    X = torch.tensor(llr, dtype=torch.float32).reshape(num_samples, 1, N)  # (B, 1, 128)
    Y = torch.tensor(info_bits, dtype=torch.float32)
    return X, Y

# ----- Training Function -----
def train_rnn_decoder(model, train_loader, val_loader, num_epochs=10, lr=0.001):
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X_batch, Y_batch in train_loader:
            X_batch, Y_batch = X_batch.cuda(), Y_batch.cuda()
            output = model(X_batch)
            loss = criterion(output, Y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for X_val, Y_val in val_loader:
                X_val, Y_val = X_val.cuda(), Y_val.cuda()
                output = model(X_val)
                val_loss = criterion(output, Y_val)
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

    return train_losses, val_losses

###########################################################



############################################################
 # BPSK modulation
    x = 1 - 2 * codewords
    snr = 10 ** (snr_db / 10)
    sigma = np.sqrt(1 / (2 * snr))
    noise = sigma * np.random.randn(*x.shape)
    y = x + noise
    llr = 2 * y / (sigma ** 2)

    # Convert to torch tensors
    X = torch.tensor(llr, dtype=torch.float32).reshape(num_samples, 1, N)  # (B, 1, 128)
    Y = torch.tensor(info_bits, dtype=torch.float32)
    return X, Y



############################################################



# --- Evaluation ---
def evaluate_decoder(decoder_func, frozen_bits_mask, snr_db, num_frames):
    total_bits = 0
    total_bit_errors = 0
    total_blocks = 0
    total_block_errors = 0
    for _ in range(num_frames):
        info_bits = np.random.randint(0, 2, K)
        tx_bits = crc_encode(info_bits)
        tx_bits = tx_bits[:K+CRC_LENGTH]
        coded_bits = polar_encode(tx_bits, frozen_bits_mask)
        modulated = bpsk_mod(coded_bits)
        noisy, sigma = add_awgn_noise(modulated, snr_db)
        llr = llr_from_awgn(noisy, sigma)
        decoded = decoder_func(llr)
        decoded_info = decoded[~frozen_bits_mask]
        total_bits += K
        total_bit_errors += np.sum(decoded_info != tx_bits)
        total_blocks += 1
        total_block_errors += int(np.any(decoded_info != tx_bits))
    ber = total_bit_errors / total_bits
    bler = total_block_errors / total_blocks
    return ber, bler

# --- Main ---
def main():
    print(f"Device: {device}")

    frozen_bits_mask = get_frozen_bits(N, K)

    scl_decoders = {L: SCLDecoder(N, K, frozen_bits_mask, L) for L in LIST_SIZES}

    rnn = RNNDecoder().to(device)
    optimizer = optim.Adam(rnn.parameters(), lr=0.001)
    criterion = nn.BCELoss()

    print("Generating training data...")
  #  X_train, Y_train = generate_training_data(50000, frozen_bits_mask)
    X_train, Y_train = generate_training_data(N, K, frozen_bits_mask, num_samples=50000, snr_db=2.0)

    print("Training RNN...")
    train_losses, val_losses = train_rnn(rnn, optimizer, criterion, X_train, Y_train, EPOCHS, BATCH_SIZE)

    ber_sc_list = []
    bler_sc_list = []
    ber_scl_lists = {L: [] for L in LIST_SIZES}
    bler_scl_lists = {L: [] for L in LIST_SIZES}
    ber_rnn_list = []
    bler_rnn_list = []

    rnn.eval()
    with torch.no_grad():
        for snr_db in snr_range:
            def sc_dec(llr):
                return sc_decode(llr, frozen_bits_mask)

            def rnn_dec(llr):
                inp = torch.tensor(llr, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
                out = rnn(inp)
                bits = (out.cpu().numpy() > 0.5).astype(int).flatten()
                return bits

            ber_sc, bler_sc = evaluate_decoder(sc_dec, frozen_bits_mask, snr_db, NUM_FRAMES)
            ber_sc_list.append(ber_sc)
            bler_sc_list.append(bler_sc)

            ber_rnn, bler_rnn = evaluate_decoder(rnn_dec, frozen_bits_mask, snr_db, NUM_FRAMES)
            ber_rnn_list.append(ber_rnn)
            bler_rnn_list.append(bler_rnn)

            for L in LIST_SIZES:
                def scl_dec(llr):
                    return scl_decoders[L].decode(llr)
                ber_scl, bler_scl = evaluate_decoder(scl_dec, frozen_bits_mask, snr_db, NUM_FRAMES)
                ber_scl_lists[L].append(ber_scl)
                bler_scl_lists[L].append(bler_scl)

            print(f"SNR={snr_db:.1f} dB | BER SC={ber_sc:.4e}, BER RNN={ber_rnn:.4e}")
            for L in LIST_SIZES:
                print(f"          | BER SCL(L={L})={ber_scl_lists[L][-1]:.4e}")
            print(f"          | BLER SC={bler_sc:.4e}, BLER RNN={bler_rnn:.4e}")
            for L in LIST_SIZES:
                print(f"          | BLER SCL(L={L})={bler_scl_lists[L][-1]:.4e}")

    # Plot Training Loss
    plt.figure(figsize=(8, 4))
    plt.plot(train_losses, label='Training Loss')
    if val_losses:
        plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('RNN Training Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Plot BER
    plt.figure(figsize=(10, 6))
    plt.yscale('log')
    plt.ylim(1e-6, 1)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Bit Error Rate (BER)')
    plt.title('BER vs SNR')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.plot(snr_range, ber_sc_list, 'o-', label='SC')
    for L in LIST_SIZES:
        plt.plot(snr_range, ber_scl_lists[L], label=f'SCL L={L}')
    plt.plot(snr_range, ber_rnn_list, 's--', label='RNN')
    plt.legend()
    plt.show()

    # Plot BLER
    plt.figure(figsize=(10, 6))
    plt.yscale('log')
    plt.ylim(1e-6, 1)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Block Error Rate (BLER)')
    plt.title('BLER vs SNR')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.plot(snr_range, bler_sc_list, 'o-', label='SC')
    for L in LIST_SIZES:
        plt.plot(snr_range, bler_scl_lists[L], label=f'SCL L={L}')
    plt.plot(snr_range, bler_rnn_list, 's--', label='RNN')
    plt.legend()
    plt.show()

if __name__ == "__main__":
    main()

Device: cuda
Generating training data...
Training RNN...


/tmp/ipython-input-1-3636049996.py:231: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32).unsqueeze(-1),
/tmp/ipython-input-1-3636049996.py:232: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(Y_train, dtype=torch.float32))


ValueError: GRU: Expected input to be 2D or 3D, got 4D instead